In [87]:
"""
TEST POWER AND AMPLITUDE BRAINPIPE FUNCTIONS
"""
import numpy as np
from os import path, listdir, makedirs
from itertools import product
from brainpipe.system import study
from brainpipe.feat.utils._feat import _manageWindow
from brainpipe.feature import power, amplitude
from rectification_smooth import rectification
import matplotlib.pyplot as plt
from scipy import stats

st = study('Olfacto')
PATH_DATA = path.join(st.path, 'database/Encoding_By_Cond_v=1_elecs=all/')
filename = path.join(PATH_DATA, '{}_odor_{}_E.npz')
subjects = ['LEFC']
channels = ['b2-b1']
conds = ['low']
meth = '2_hilb'

################################################################################################
sf = 512
widths = [None, 128, 256, 358] #no downsampling, 250ms, 500ms, 700ms
steps = [5, 51] #steps None, 10ms, 100ms
################################################################################################

for su,cond in product(subjects,conds):
    mat = np.load(filename.format(su,cond),allow_pickle=True)
    chs_idx = [i for i,ch in enumerate(mat['channels']) if ch in channels]
    data = mat['x'][chs_idx,...]
    nelec, npts, ntrials = data.shape
    
    for t in range(ntrials):
        fig, axs = plt.subplots(8,4, figsize=(20, 40), facecolor='w', edgecolor='k')
        fig.subplots_adjust(hspace = .5, wspace=.001)
        axs = axs.ravel()
        i = 0
        for width,step in product(widths,steps):
            kwargs = {} # Define an empty dictionnary to saveall power parameters
            kwargs['f'] = [[3,7]] # Frequency vector
            fname = ['theta'] # Name of each frequencykwargs['f'] = [[3,7], [8,13], [13,30], [30,120]] # Frequency vector[0.1,1.5],[2,3]
            kwargs['split'] = None
            kwargs['width'], kwargs['step'] = width, step # take power in 358 samples (700ms) windows width every 51 samples (100ms)

            # Compute and save power objects :
            #ampObj_x = amplitude(sf, data.shape[1],baseline=None,**kwargs, method=meth)
            powObj_x = power(sf, data.shape[1],baseline=None,**kwargs, method='hilbert')
            rect_x = rectification(data,sf,[3,7],width,step)
            powObj_x1 = power(sf, data.shape[1],baseline=None,**kwargs, method='hilbert1')
            win_all, time = _manageWindow(data.shape[1], width=width, step=step)
            #amp_ = ampObj_x.get(data,n_jobs=-1)[0]
            pow_ = powObj_x.get(data,n_jobs=-1)[0]
            pow_1 = powObj_x1.get(data,n_jobs=-1)[0]

            time = (np.array(time)/sf)-3
            nsel = [i for i,t in enumerate(time) if -1<=t<=2]
            time_sel = time[nsel]
            pow_sel, pow_sel1 = rect_x[0,:,nsel,2], pow_1[0,:,nsel,2]
            pow_sel2 = pow_[0,:,nsel,2]
            print(pow_sel.shape,pow_sel1.shape,data.shape)
            R, p = stats.pearsonr(np.ravel(pow_sel),np.ravel(pow_sel1))
            print(width, step, R,p)
            #amp_sel = amp_[0,:,nsel,0]

            #plot power enveloppe (trial1)
            axs[i].plot(time,rect_x[0,0,:,2],'green')
            axs[i].set_title('Rectif width={}, step={}'.format(width,step))
            axs[i+1].plot(time,pow_1[0,0,:,2],'blue')
            axs[i+1].set_title('BAD HILBERT width={}, step={}'.format(width,step))
            axs[i+2].plot(time,pow_[0,0,:,2],'black')
            axs[i+2].set_title('HILBERT width={}, step={}'.format(width,step))
            axs[i+3].plot(np.arange(3584),data[0,:,2],'black')
            axs[i+3].set_title('RAW width={}, step={}'.format(width,step))
            i += 4
        #plt.show()
        plt.savefig('theta_power_example_smoothing_'+meth+'_trial='+str(t)+'.png')
    0/0

-> Olfacto loaded
(1, 3584, 23)


ZeroDivisionError: division by zero

In [64]:
512*3

1536

In [ ]:
"""
COMPUTE POWER WITH DIFFERENT STEP AND WIDTH
"""

"""
TEST POWER AND AMPLITUDE BRAINPIPE FUNCTIONS
"""
import numpy as np
from os import path, listdir, makedirs
from itertools import product
from brainpipe.system import study
from brainpipe.feat.utils._feat import _manageWindow
from brainpipe.feature import power, amplitude
import matplotlib.pyplot as plt

st = study('Olfacto')
PATH_DATA = path.join(st.path, 'database/Encoding_By_Cond_v=1_elecs=all/')
filename = path.join(PATH_DATA, '{}_odor_{}_E.npz')
savename = path.join(st.path, 'feature/TPSim_power_data/try_smooth/')
subjects = ['LEFC','CHAF','FERJ']
channels = ['o7-o6']
conds = ['low']

################################################################################################
sf = 512
widths = [128, 256, 358] #no downsampling, 250ms, 500ms, 700ms
steps = [5, 51] #steps None, 10ms, 100ms
################################################################################################

for su,cond in product(subjects,conds):
    mat = np.load(filename.format(su,cond),allow_pickle=True)
    
    for width,step in product(widths,steps):
        kwargs = {} # Define an empty dictionnary to saveall power parameters
        kwargs['f'] = [[3,7]] # Frequency vector
        fname = ['theta'] # Name of each frequencykwargs['f'] = [[3,7], [8,13], [13,30], [30,120]] # Frequency vector[0.1,1.5],[2,3]
        kwargs['split'] = None
        kwargs['width'], kwargs['step'] = width, step # take power in 358 samples (700ms) windows width every 51 samples (100ms)

        # Compute and save power objects :
        powObj_x = power(sf, data.shape[1],baseline=None,**kwargs, method='hilbert1')
        win_all, time = _manageWindow(data.shape[1], width=width, step=step)
        pow_ = powObj_x.get(data,n_jobs=-1)[0]
